In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../')

import os

os.environ['MKL_NUM_THREADS'] = '4'
os.environ['GOTO_NUM_THREADS'] = '4'
os.environ['OMP_NUM_THREADS'] = '4'

import theano
theano.config.openmp = True

os.environ['THEANO_FLAGS'] = 'exception_verbosity=high,device=cpu,blas.ldflags=-lblas -lgfortran'

import cPickle as pickle

import json
from copy import copy

from itertools import groupby,chain,tee,izip,islice
from collections import Iterable,Counter
from operator import itemgetter 

import numpy as np
from random import shuffle

from ipywidgets import interact

from sklearn.cross_validation import train_test_split

import re

import nltk
from nltk.stem import RSLPStemmer,SnowballStemmer
from nltk import PunktSentenceTokenizer,FreqDist
from nltk.corpus import stopwords
from nltk import UnigramTagger,BigramTagger

from gensim import corpora

from keras.models import Sequential,model_from_json  
from keras.layers.core import TimeDistributedDense,RepeatVector, Activation, Dropout,Dense,Flatten
from keras.layers.recurrent import GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint    
# from keras.utils.visualize_util import plot
import seq2seq
from seq2seq.models import SimpleSeq2seq

from utils_pack.utils import pickle_in,ensure_dir
from utils_pack.word_embeddings_utils import IngredientDataTransformer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
X_test,Y_test = pickle_in(os.path.join("../datasets","deep_learning_datasets","test.pkl"))
dictionary = pickle_in(os.path.join("../datasets","deep_learning_datasets","dictionary.pkl"))

In [3]:
model_name = "model_seq2seq_1"
model_folder = "../models"

model = pickle_in(os.path.join(model_folder,model_name,'model_architecture.json'))
# json_file = open(os.path.join(model_folder,model_name,"model_architecture.json"), 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json,custom_objects={"SimpleSeq2seq":SimpleSeq2seq})
# load weights into new model
model.load_weights(os.path.join(model_folder,model_name,"model_weights.h5"))

In [22]:
a = model.predict(X_test)

MissingInputError: ("An input of the graph, used to compute DimShuffle{x,x}(keras_learning_phase), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", keras_learning_phase)

In [20]:
def predict_text_ingredient(model,x,y):


    ingredients_text = []
    for i in range(x.shape[0]):
        if x[i].tolist() != [0]*x.shape[0]:
            ingredients_text.append(" ".join([dictionary[j-1] for j in x[i] if j!=0]))
    ingredients_text = "\n".join(ingredients_text)
    ground_truth_text =" ".join([dictionary[np.argmax(word)-1] for word in y if np.argmax(word)!=0])
    print "ingr",ingredients_text 
    print "truth",ground_truth_text
    prediction = model.predict_classes(x.reshape(1,x.shape[0],x.shape[1])) 
    prediction_text = " ".join([dictionary[i-1] for i in prediction[0] if i!=0])
    return (ingredients_text,ground_truth_text,prediction_text)

In [21]:
def present_results(nr):
    nr = int(nr)
    x = X_test[nr]
    y = Y_test[nr]
    
    ingredients_text,ground_truth_text,prediction_text = predict_text_ingredient(model,x,y)
    print "INGREDIENTS LIST:\n",ingredients_text
    print "\nGROUND TRUTH:\n",ground_truth_text
    print "\nPREDICTION:\n",prediction_text
    
interact(present_results,nr="10")

ingr 1 lata de leite condensado
4 ovos inteiros
Nozes moídas
1 cálice de vinho
truth mesma medida de leite de vaca


MissingInputError: ("An input of the graph, used to compute DimShuffle{x,x}(keras_learning_phase), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.", keras_learning_phase)

<function __main__.present_results>